## Importando Utilitários

In [218]:
import numpy as np

## Criando Classe Tensor

In [219]:
class Tensor():

    def __init__(self, value, requires_grad=False):

        self.value = np.array(value, dtype=np.float64)

        self.requires_grad = requires_grad

        self.grad_fn = None

        self.grad = None

        self.children = []

    def __add__(self, other):

        result = Tensor(self.value + other.value)

        def add_backward():

            if result.grad is None:

                return RuntimeError("result.grad is None.")

            if self.grad is None:

                self.grad = (result.grad.copy())

            else:

                self.grad += (result.grad.copy())

            if other.grad is None:

                other.grad = (result.grad.copy())

            else:

                other.grad += (result.grad.copy())
        
        if self.requires_grad or other.requires_grad:

            result.requires_grad = True

            result.grad_fn = add_backward

            if self.requires_grad:
    
                result.children.append(self)
    
            if other.requires_grad:
    
                result.children.append(other)

        return result

    def __sub__(self, other):

        result = Tensor(self.value - other.value)

        def sub_backward():

            if result.grad is None:

                return RuntimeError("result.grad is None.")

            if self.grad is None:

                self.grad = (result.grad.copy())

            else:

                self.grad += (result.grad.copy())

            if other.grad is None:

                other.grad = (-result.grad.copy())

            else:

                other.grad -= (result.grad.copy())

        
        if self.requires_grad or other.requires_grad:

            result.requires_grad = True

            result.grad_fn = sub_backward

            if self.requires_grad:
    
                result.children.append(self)
    
            if other.requires_grad:
    
                result.children.append(other)

        return result

    def __mul__(self, other):

        result = Tensor(self.value * other.value)

        def mul_backward():

            if result.grad is None:

                return RuntimeError("result.grad is None.")

            if self.grad is None:

                self.grad = (result.grad.copy() * other.value)

            else:

                self.grad += (result.grad.copy() * other.value)

            if other.grad is None:

                other.grad = (result.grad.copy() * self.value)

            else:

                other.grad += (result.grad.copy() * self.value)

        
        if self.requires_grad or other.requires_grad:

            result.requires_grad = True

            result.grad_fn = mul_backward

            if self.requires_grad:
    
                result.children.append(self)
    
            if other.requires_grad:
    
                result.children.append(other)

        return result

    def __truediv__(self, other):

        result = Tensor(self.value / other.value)

        def div_backward():

            if result.grad is None:

                return RuntimeError("result.grad is None.")

            if self.grad is None:

                self.grad = (result.grad.copy() * (np.float64(1.0) / other.value.copy()))

            else:

                self.grad += (result.grad.copy() * (np.float64(1.0) / other.value.copy()))

            if other.grad is None:

                other.grad = (result.grad.copy() * (-self.value.copy() * (other.value.copy() ** np.float64(-2.0))))

            else:

                other.grad += (result.grad.copy() * (-self.value.copy() * (other.value.copy() ** np.float64(-2.0))))

        
        if self.requires_grad or other.requires_grad:

            result.requires_grad = True

            result.grad_fn = div_backward

            if self.requires_grad:
    
                result.children.append(self)
    
            if other.requires_grad:
    
                result.children.append(other)

        return result
        
        
    def backward(self):

        if self.requires_grad is None:

            return RuntimeError("requires_grad is set to False on that Tensor.")

        if self.grad is None:

            self.grad = np.ones_like(self.value)

        if self.grad_fn is not None:

            self.grad_fn()
        
        for child in self.children:

            child.backward()

    def __str__(self):

        return "Tensor({}, requires_grad={}, grad_fn={})".format(self.value, self.requires_grad, self.grad_fn.__name__ if self.grad_fn != None else None)

    def __repr__(self):

        return "Tensor({}, requires_grad={}, grad_fn={})".format(self.value, self.requires_grad, self.grad_fn.__name__ if self.grad_fn != None else None)       

## Computando Gradientes

In [220]:
a = Tensor([20.0,30.0,40.0], requires_grad=True)

b = Tensor([50.0, 60.0, 70.0], requires_grad=True)

c = ((a + b) / (a - b)) * ((a + b) / (a - b))

c.backward()

a.grad, b.grad, c.grad

(array([0.51851852, 0.8       , 1.14074074]),
 array([-0.20740741, -0.4       , -0.65185185]),
 array([1., 1., 1.]))